In [ ]:
! pip install rouge_score
! pip install -U accelerate
!pip install evaluate

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=32537abda5f417668ba368796bd451b4d111ad06539c8cf67a168160d1f64882
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00


In [ ]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [ ]:
!pip install transformers

In [ ]:
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 7.9 MB/s eta 0:00:00


In [ ]:
import glob
from transformers import pipeline, set_seed

from datasets import load_metric#, load_from_disk

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer

from tqdm import tqdm
import torch
from datasets import Dataset
from sklearn.metrics import f1_score
import numpy as np
import os
import accelerate
import gc
from transformers import AutoConfig,BioGptTokenizer, BioGptForCausalLM
import evaluate
import nltk
from nltk.tokenize import sent_tokenize
from evaluate import load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def read_paper(path):
  f = open(path, 'r', encoding="utf-8")
  text = str(f.read())
  f.close()
  return text

In [ ]:
def create_list(folder_path):
  temp_papers = []
  for filename in tqdm(glob.glob(folder_path + "/*.txt")):
      temp_papers.append(read_paper(filename))
  for filename in tqdm(glob.glob(folder_path + "/*.xml")):
      temp_papers.append(read_paper(filename))
  return temp_papers

In [ ]:
test_path = '/content/drive/MyDrive/test'
list_test = create_list(test_path)

100%|██████████| 86/86 [00:00<00:00, 1073.58it/s]


In [ ]:
list_test[1]

"Admission Date:  [**2207-8-16**]              Discharge Date:   [**2207-9-2**]\n\nDate of Birth:  [**2150-9-2**]             Sex:   M\n\nService: SURGERY\n\nAllergies:\nNo Known Allergies / Adverse Drug Reactions\n\nAttending:[**First Name3 (LF) 1234**]\nChief Complaint:\nSevere lower extremity lymphedema with superimposed cellulitis\n\nMajor Surgical or Invasive Procedure:\n[**2207-8-24**]:Left below-knee amputation (circular guillotine).\n[**2207-8-28**]: Re-amputation and closure of left below-knee\namputation.\n\n\nHistory of Present Illness:\n56yo male with a history of DM2, CAD s/p CABG, peripheral\nvascular disease, and prior history of chronic lymphedema was\nadmitted on [**2207-8-16**] for LLE cellulitis.  His left lower\nextremity lymphedema started after vascular surgery in [**2205**] (L\nSFA to anterior tibial bypass graft).  Left leg is extremely\npainful. More recently he has been on his feet more, which has\nbeen causing worsening edema and pain. He has occasional serou

In [ ]:
list_test[-1]

'<?xml version="1.0" encoding="UTF-8" ?>\n<PatientMatching>\n<TEXT><![CDATA[\n\nRecord date: 2091-07-11\n\nReason for Visit\t\n\nComprehensive evaluation of all problems.\n\n\n\nProblems\t  \n\nhypertension\t      \n\nPRESENT SINCE MID 2060\'S. FOLLOWED BY DR. THACKER AT DARDANELLE HOSPITAL. \n\n\tETT 5/12/82 TO 98% PRMHR SHOWED ST CHANGES WHICH WERE SUGGESTIVE BUT NOT DIAGNOSTIC AND THERE WAS BASELINE LVH WITH STRAIN BUT THALLIUM WAS NEG. THERE WERE OCCASIONAL PVC\'S AND COUPLETS AND TWO 3 BEAT RUNS OF VT.     HAD BEEN ACCEPTABLE ALBEIT ON AN ELABORATE REGIMEN THAT HAD EVOLVED. SUBOPTIMAL WITH ADDITIONAL WEIGHT GAIN 5/23/86. WILL RECHECK 6-8 WEEKS AFTER 8-10 LB WEIGHT LOSS. ETT THAL SCHEDULED, GIVEN ABNL ECG, RISK FACTORS, AND HIS CONTINUING DESIRE TO EXERCISE. \n\n\t5/86 EXCELLENT EXERCISE CAPACITY TO 9 METS, ECG NONDIAGNOSTIC DUE TO BASELINE ABNORMALITIES BUT THALLIUM NL.     \n\n\t150/74 7/86 AND WILL ACCEPT THIS FOR NOW BUT STILL NEEDS TO LOSE WEIGHT. \n\n\t140/78 9/86 WO WEIGHT L

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
def batch_generator(data, batch_size):
    s = 0
    e = s + batch_size
    while s < len(data):
        yield data[s:e]
        s = e
        e = min(s + batch_size, len(data))


def summarized_text(model, articles, tokenizer, batch_size: int = 16):

    def perform_inference(batch: list) -> list:
        inputs = tokenizer(
            batch, max_length=511, return_tensors="pt", padding=True, truncation=True
        )

        summary_ids = model.generate(
            inputs.input_ids.to(device),
            attention_mask=inputs.attention_mask.to(device),
            num_beams=2,
            min_length=0,
            max_length=512,
        )
        return tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

    res = []

    summary_articles = list(map(lambda article:article, articles))
    for batch in batch_generator(summary_articles, batch_size=batch_size):
        res += perform_inference(batch)

        torch.cuda.empty_cache()
        gc.collect()

    # clean up
    del tokenizer
    del model
    torch.cuda.empty_cache()
    gc.collect()
    return res

In [ ]:
def summarized_text1(model, articles, tokenizer, batch_size: int = 16):

    def perform_inference(batch: list) -> list:
        inputs = tokenizer(
            batch, max_length=511, return_tensors="pt", padding=True, truncation=True
        )

        summary_ids = model.predict(
            inputs.input_ids.to(device),
            attention_mask=inputs.attention_mask.to(device),
            num_beams=2,
            min_length=0,
            max_length=512,
        )
        return tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

    res = []

    summary_articles = list(map(lambda article:article, articles))
    for batch in batch_generator(summary_articles, batch_size=batch_size):
        res += perform_inference(batch)

        torch.cuda.empty_cache()
        gc.collect()

    # clean up
    del tokenizer
    del model
    torch.cuda.empty_cache()
    gc.collect()
    return res

In [ ]:
def write_summary(original_summary, trained_summary, model_name):
  print("the model is:",model_name)
  if model_name == 'CompactBioBert':
    original_path = "/content/drive/MyDrive/models/Compact_BioBERT/Summary.txt"
    trained_path = "/content/drive/MyDrive/models/Compact_BioBERT/Summary_trained.txt"
  elif model_name == 'SciBert':
    original_path = "/content/drive/MyDrive/models/SciBERT/Summary.txt"
    trained_path = "/content/drive/MyDrive/models/SciBERT/Summary_trained.txt"
  elif model_name == 'ClinicalBioBert':
    original_path = "/content/drive/MyDrive/models/ClinicalBioBERT/Summary.txt"
    trained_path = "/content/drive/MyDrive/models/ClinicalBioBERT/Summary_trained.txt"
  elif model_name == 'ClinicalLongformer':
    original_path = "/content/drive/MyDrive/models/ClinicalLongformer/Summary.txt"
    trained_path = "/content/drive/MyDrive/models/ClinicalLongformer/Summary_trained.txt"
  elif model_name == 'BioGPT':
    original_path = "/content/drive/MyDrive/models/Biogpt/Summary.txt"
    trained_path = "/content/drive/MyDrive/models/Biogpt/Summary_trained.txt"
  else:
    print("no matching model found")
    return
  with open(original_path, 'w') as fp:
    for summary in original_summary:
        # write each item on a new line
        fp.write("%s\n" % summary)
    print('Original Summary Done for model: ', model_name)
  with open(trained_path, 'w') as fp:
    for summary in trained_summary:
        # write each item on a new line
        fp.write("%s\n" % summary)
    print('Trained Summary Done for model: ', model_name)

In [ ]:
model = "nlpie/compact-biobert"

tokenizer = AutoTokenizer.from_pretrained(model)

model_compactbiobert = AutoModelForMaskedLM.from_pretrained(model).to(device)

summaries_original_compact_biobert = summarized_text(model_compactbiobert,list_test,tokenizer)

model = "/content/drive/MyDrive/models/Compact_BioBERT/"

model_compactbiobert = AutoModelForMaskedLM.from_pretrained(model).to(device)

summaries_trained_compact_biobert = summarized_text(model_compactbiobert,list_test,tokenizer)

In [ ]:
nltk.download("punkt")

rouge_score = evaluate.load("rouge")

def compute_rouge_score(generated: list, reference: list):
    generated_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in generated]
    reference_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in reference]
    return rouge_score.compute(
        predictions=generated_with_newlines,
        references=reference_with_newlines,
        use_stemmer=True,
    )

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
compute_rouge_score(summaries_original_compact_biobert,list_test)

{'rouge1': 0.20122213499331798,
 'rouge2': 0.20057367725069408,
 'rougeL': 0.20141096389796484,
 'rougeLsum': 0.20127061716509814}

In [ ]:
# write_summary(summaries_original_compact_biobert,summaries_trained_compact_biobert,'CompactBioBert')

In [ ]:
model = "emilyalsentzer/Bio_ClinicalBERT"

tokenizer = AutoTokenizer.from_pretrained(model)

model_ClinicalBioBert = AutoModelForMaskedLM.from_pretrained(model).to(device)

summaries_original_ClinicalBioBert = summarized_text(model_ClinicalBioBert,list_test,tokenizer)

model = "/content/drive/MyDrive/models/ClinicalBioBERT/"

model_ClinicalBioBert = AutoModelForMaskedLM.from_pretrained(model).to(device)

summaries_trained_ClinicalBioBert = summarized_text(model_ClinicalBioBert,list_test,tokenizer)

In [ ]:
# write_summary(summaries_original_ClinicalBioBert,summaries_trained_ClinicalBioBert,'ClinicalBioBert')

In [ ]:
model = "allenai/scibert_scivocab_uncased"

tokenizer = AutoTokenizer.from_pretrained(model)

model_SciBert = AutoModelForMaskedLM.from_pretrained(model).to(device)

summaries_original_SciBert = summarized_text(model_SciBert,list_test,tokenizer)

model = "/content/drive/MyDrive/models/SciBERT/"

model_SciBert = AutoModelForMaskedLM.from_pretrained(model).to(device)

summaries_trained_SciBert = summarized_text(model_SciBert,list_test,tokenizer)

In [ ]:
# write_summary(summaries_original_SciBert,summaries_trained_SciBert,'SciBert')

In [ ]:
model = "microsoft/biogpt"

tokenizer = BioGptTokenizer.from_pretrained(model)

model_biogpt = BioGptForCausalLM.from_pretrained(model).to(device)

summaries_original_biogpt = summarized_text(model_biogpt,list_test,tokenizer)

model = "/content/drive/MyDrive/models/Biogpt/"

model_biogpt = BioGptForCausalLM.from_pretrained(model).to(device)

summaries_trained_biogpt = summarized_text(model_biogpt,list_test,tokenizer)

In [ ]:
# write_summary(summaries_original_biogpt,summaries_trained_biogpt,'BioGPT')

In [ ]:
# # Original Longformer
# model = "yikuan8/Clinical-Longformer"

# tokenizer = AutoTokenizer.from_pretrained(model)

# model_Longformer = AutoModelForMaskedLM.from_pretrained(model).to(device)

# summaries_original_Longformer = summarized_text(model_Longformer,list_test,tokenizer)

# # Trained Longformer
# model = "/content/drive/MyDrive/models/ClinicalLongformer/"

# model_Longformer = AutoModelForMaskedLM.from_pretrained(model).to(device)

# summaries_trained_Longformer = summarized_text1(model_Longformer,list_test,tokenizer)

In [ ]:
# write_summary(summaries_original_Longformer,summaries_trained_Longformer,'ClinicalLongformer')

In [ ]:
summaries_original_compact_biobert[0]

"Admission Date : [ * * 2123 - 11 - 4 * * ] Discharge Date : [ * * 2123 - 11 - 16 * * ] Date of Birth : [ * * 2044 - 3 - 15 * * ] Sex : F Service : MEDICINE Allergies : Penicillins / Inderal / Morphine Attending : [ * * First Name3 ( LF ) 898 * * ] Chief Complaint : post - op bleeding after tooth extraction Major Surgical or Invasive Procedure : None History of Present Illness : 79 yo woman with history of a - fib, CHF s / p bioprosthetic mitral and s / p tricuspid valve annuloplasty, who initially was admitted to [ * * 11 - 4 * * ] after she presented to ED with persistent bleeding following L tooth extractions and posterior superior palate I and D. Patient initially had INR elevated to 2. 1 secondary to warfarin use for atrial fibrillation. In ED ENT was consulted and pt was packed to help control bleeding. Her INR was reversed with vitamin K, 2 units FFP and her INR now is 1. 1. She was hypotensive with SBP in 80's on admission and was admitted to the ICU. Hct was 21 on admission ( 

In [ ]:
summaries_trained_compact_biobert[0]

"Admission Date : [ * * 2123 - 11 - 4 * * ] Discharge Date : [ * * 2123 - 11 - 16 * * ] Date of Birth : [ * * 2044 - 3 - 15 * * ] Sex : F Service : MEDICINE Allergies : Penicillins / Inderal / Morphine Attending : [ * * First Name3 ( LF ) 898 * * ] Chief Complaint : post - op bleeding after tooth extraction Major Surgical or Invasive Procedure : None History of Present Illness : 79 yo woman with history of a - fib, CHF s / p bioprosthetic mitral and s / p tricuspid valve annuloplasty, who initially was admitted to [ * * 11 - 4 * * ] after she presented to ED with persistent bleeding following L tooth extractions and posterior superior palate I and D. Patient initially had INR elevated to 2. 1 secondary to warfarin use for atrial fibrillation. In ED ENT was consulted and pt was packed to help control bleeding. Her INR was reversed with vitamin K, 2 units FFP and her INR now is 1. 1. She was hypotensive with SBP in 80's on admission and was admitted to the ICU. Hct was 21 on admission ( 

In [ ]:
compute_rouge_score(summaries_original_compact_biobert,list_test)

{'rouge1': 0.20122213499331798,
 'rouge2': 0.20057367725069408,
 'rougeL': 0.20141096389796484,
 'rougeLsum': 0.20127061716509814}

In [ ]:
compute_rouge_score(summaries_trained_compact_biobert,list_test)

{'rouge1': 0.20122221553818245,
 'rouge2': 0.20056875821432546,
 'rougeL': 0.20141504139995908,
 'rougeLsum': 0.2012739664491917}

In [ ]:
compute_rouge_score(summaries_original_ClinicalBioBert,list_test)

{'rouge1': 0.21196020221573364,
 'rouge2': 0.21130198446013643,
 'rougeL': 0.21223295247165702,
 'rougeLsum': 0.21229613057225694}

In [ ]:
compute_rouge_score(summaries_trained_ClinicalBioBert,list_test)

{'rouge1': 0.21195168558211636,
 'rouge2': 0.2113015442038068,
 'rougeL': 0.21220128666468324,
 'rougeLsum': 0.2122892398756217}

In [ ]:
compute_rouge_score(summaries_original_SciBert,list_test)

{'rouge1': 0.23204505230208183,
 'rouge2': 0.23156052882938538,
 'rougeL': 0.2323396271291559,
 'rougeLsum': 0.23233918686510066}

In [ ]:
compute_rouge_score(summaries_trained_SciBert,list_test)

{'rouge1': 0.23202867298741012,
 'rouge2': 0.2315696816714135,
 'rougeL': 0.23232341531659068,
 'rougeLsum': 0.2323002365372667}

In [ ]:
compute_rouge_score(summaries_original_biogpt,list_test)

{'rouge1': 0.22771826297685882,
 'rouge2': 0.22711546891351045,
 'rougeL': 0.22805123595990917,
 'rougeLsum': 0.22792399638403738}

In [ ]:
compute_rouge_score(summaries_trained_biogpt,list_test)

{'rouge1': 0.22775466977173958,
 'rouge2': 0.2270923441622205,
 'rougeL': 0.2280893206423023,
 'rougeLsum': 0.22795569754873857}

In [ ]:
#compute_rouge_score(summaries_original_Longformer,list_test)

In [ ]:
#compute_rouge_score(summaries_trained_Longformer,list_test)

In [ ]:
bertscore = load("bertscore")

In [ ]:
results_compact_biobert = bertscore.compute(predictions=summaries_original_compact_biobert, references=list_test, lang="en")
precision_orig_compactbiobert = sum(results_compact_biobert['precision'])/len(list_test)
recall_orig_compactbiobert = sum(results_compact_biobert['recall'])/len(list_test)
f1_orig_compactbiobert = sum(results_compact_biobert['f1'])/len(list_test)
print("Original Compact Biobert precision =", precision_orig_compactbiobert)
print("Original Compact Biobert recall =", recall_orig_compactbiobert)
print("Original Compact Biobert f1 score =", f1_orig_compactbiobert)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Original Compact Biobert precision = 0.9144040885425749
Original Compact Biobert recall = 0.8896430180186317
Original Compact Biobert f1 score = 0.9017809487524486


In [ ]:
results_trained_compact_biobert = bertscore.compute(predictions=summaries_trained_compact_biobert, references=list_test, lang="en")
precision_trained_compactbiobert = sum(results_trained_compact_biobert['precision'])/len(list_test)
recall_trained_compactbiobert = sum(results_trained_compact_biobert['recall'])/len(list_test)
f1_trained_compactbiobert = sum(results_trained_compact_biobert['f1'])/len(list_test)
print("trained Compact Biobert precision =", precision_trained_compactbiobert)
print("trained Compact Biobert recall =", recall_trained_compactbiobert)
print("trained Compact Biobert f1 score =", f1_trained_compactbiobert)

trained Compact Biobert precision = 0.9143557622319176
trained Compact Biobert recall = 0.889650285243988
trained Compact Biobert f1 score = 0.9017607376689003


In [ ]:
results_orig_clinicalbiobert = bertscore.compute(predictions=summaries_original_ClinicalBioBert, references=list_test, lang="en")
precision_orig_clinicalbiobert = sum(results_orig_clinicalbiobert['precision'])/len(list_test)
recall_orig_clinicalbiobert = sum(results_orig_clinicalbiobert['recall'])/len(list_test)
f1_orig_clinicalbiobert = sum(results_orig_clinicalbiobert['f1'])/len(list_test)
print("Original Clinical Biobert precision =", precision_orig_clinicalbiobert)
print("Original Clinical Biobert recall =", recall_orig_clinicalbiobert)
print("Original Clinical Biobert f1 score =", f1_orig_clinicalbiobert)

Original Clinical Biobert precision = 0.8906928522246225
Original Clinical Biobert recall = 0.8718619607743763
Original Clinical Biobert f1 score = 0.8811092297236125


In [ ]:
results_trained_clinicalbiobert = bertscore.compute(predictions=summaries_trained_ClinicalBioBert, references=list_test, lang="en")
precision_trained_clinicalbiobert = sum(results_trained_clinicalbiobert['precision'])/len(list_test)
recall_trained_clinicalbiobert = sum(results_trained_clinicalbiobert['recall'])/len(list_test)
f1_trained_clinicalbiobert = sum(results_trained_clinicalbiobert['f1'])/len(list_test)
print("Original Clinical Biobert precision =", precision_trained_clinicalbiobert)
print("Original Clinical Biobert recall =", recall_trained_clinicalbiobert)
print("Original Clinical Biobert f1 score =", f1_trained_clinicalbiobert)

Original Clinical Biobert precision = 0.89067633833204
Original Clinical Biobert recall = 0.8718954001154219
Original Clinical Biobert f1 score = 0.8811186376072112


In [ ]:
print("Trained Clinical Biobert precision =", precision_trained_clinicalbiobert)
print("Trained Clinical Biobert recall =", recall_trained_clinicalbiobert)
print("Treained Clinical Biobert f1 score =", f1_trained_clinicalbiobert)

Trained Clinical Biobert precision = 0.89067633833204
Trained Clinical Biobert recall = 0.8718954001154219
Treained Clinical Biobert f1 score = 0.8811186376072112


In [ ]:
results_orig_scibert = bertscore.compute(predictions=summaries_original_SciBert, references=list_test, lang="en")
precision_orig_scibert = sum(results_orig_scibert['precision'])/len(list_test)
recall_orig_scibert = sum(results_orig_scibert['recall'])/len(list_test)
f1_orig_scibert = sum(results_orig_scibert['f1'])/len(list_test)
print("original scibert precision =", precision_orig_scibert)
print("original scibert recall =", recall_orig_scibert)
print("original scibert f1 score =", f1_orig_scibert)

original scibert precision = 0.8816167405673436
original scibert recall = 0.8710462927818299
original scibert f1 score = 0.8762273362704686


In [ ]:
results_trained_scibert = bertscore.compute(predictions=summaries_trained_SciBert, references=list_test, lang="en")
precision_trained_scibiobert = sum(results_trained_scibert['precision'])/len(list_test)
recall_trained_scibiobert = sum(results_trained_scibert['recall'])/len(list_test)
f1_trained_scibiobert = sum(results_trained_scibert['f1'])/len(list_test)
print("trained scibert precision =", precision_trained_scibiobert)
print("trained scibert recall =", recall_trained_scibiobert)
print("trained scibert f1 score =", f1_trained_scibiobert)

trained scibert precision = 0.8816167405673436
trained scibert recall = 0.8710462927818299
trained scibert f1 score = 0.8762273362704686


In [ ]:
results_orig_biogpt = bertscore.compute(predictions=summaries_original_biogpt, references=list_test, lang="en")
precision_orig_biogpt = sum(results_orig_biogpt['precision'])/len(list_test)
recall_orig_biogpt = sum(results_orig_biogpt['recall'])/len(list_test)
f1_orig_biogpt = sum(results_orig_biogpt['f1'])/len(list_test)
print("original biogpt precision =", precision_orig_biogpt)
print("original biogpt recall =", recall_orig_biogpt)
print("original biogpt f1 score =", f1_orig_biogpt)

original biogpt precision = 0.9039299885431925
original biogpt recall = 0.8903449620519366
original biogpt f1 score = 0.8970168630282084


In [ ]:
results_trained_biogpt = bertscore.compute(predictions=summaries_trained_biogpt, references=list_test, lang="en")
precision_trained_biogpt = sum(results_trained_biogpt['precision'])/len(list_test)
recall_trained_biogpt = sum(results_trained_biogpt['recall'])/len(list_test)
f1_trained_biogpt = sum(results_trained_biogpt['f1'])/len(list_test)
print("trained biogpt precision =", precision_trained_biogpt)
print("trained biogpt recall =", recall_trained_biogpt)
print("trained biogpt f1 score =", f1_trained_biogpt)

trained biogpt precision = 0.903920681135995
trained biogpt recall = 0.8903503287406195
trained biogpt f1 score = 0.8970149676005046


In [ ]:
print(list_test[1])


Admission Date:  [**2207-8-16**]              Discharge Date:   [**2207-9-2**]

Date of Birth:  [**2150-9-2**]             Sex:   M

Service: SURGERY

Allergies:
No Known Allergies / Adverse Drug Reactions

Attending:[**First Name3 (LF) 1234**]
Chief Complaint:
Severe lower extremity lymphedema with superimposed cellulitis

Major Surgical or Invasive Procedure:
[**2207-8-24**]:Left below-knee amputation (circular guillotine).
[**2207-8-28**]: Re-amputation and closure of left below-knee
amputation.


History of Present Illness:
56yo male with a history of DM2, CAD s/p CABG, peripheral
vascular disease, and prior history of chronic lymphedema was
admitted on [**2207-8-16**] for LLE cellulitis.  His left lower
extremity lymphedema started after vascular surgery in [**2205**] (L
SFA to anterior tibial bypass graft).  Left leg is extremely
painful. More recently he has been on his feet more, which has
been causing worsening edema and pain. He has occasional serous
drainage from left leg.  

In [ ]:
print(summaries_original_compact_biobert[1])

Admission Date : [ * * 2207 - 8 - 16 * * ] Discharge Date : [ * * 2207 - 9 - 2 * * ] Date of Birth : [ * * 2150 - 9 - 2 * * ] Sex : M Service : SURGERY Allergies : No Known Allergies / Adverse Drug Reactions Attending : [ * * First Name3 ( LF ) 1234 * * ] Chief Complaint : Severe lower extremity lymphedema with superimposed cellulitis Major Surgical or Invasive Procedure : [ * * 2207 - 8 - 24 * * ] : Left below - knee amputation ( circular guillotine ). [ * * 2207 - 8 - 28 * * ] : Re - amputation and closure of left below - knee amputation. History of Present Illness : 56yo male with a history of DM2, CAD s / p CABG, peripheral vascular disease, and prior history of chronic lymphedema was admitted on [ * * 2207 - 8 - 16 * * ] for LLE cellulitis. His left lower extremity lymphedema started after vascular surgery in [ * * 2205 * * ] ( L SFA to anterior tibial bypass graft ). Left leg is extremely painful. More recently he has been on his feet more, which has been causing worsening edema 

In [ ]:
print(summaries_trained_compact_biobert[1])

Admission Date : [ * * 2207 - 8 - 16 * * ] Discharge Date : [ * * 2207 - 9 - 2 * * ] Date of Birth : [ * * 2150 - 9 - 2 * * ] Sex : M Service : SURGERY Allergies : No Known Allergies / Adverse Drug Reactions Attending : [ * * First Name3 ( LF ) 1234 * * ] Chief Complaint : Severe lower extremity lymphedema with superimposed cellulitis Major Surgical or Invasive Procedure : [ * * 2207 - 8 - 24 * * ] : Left below - knee amputation ( circular guillotine ). [ * * 2207 - 8 - 28 * * ] : Re - amputation and closure of left below - knee amputation. History of Present Illness : 56yo male with a history of DM2, CAD s / p CABG, peripheral vascular disease, and prior history of chronic lymphedema was admitted on [ * * 2207 - 8 - 16 * * ] for LLE cellulitis. His left lower extremity lymphedema started after vascular surgery in [ * * 2205 * * ] ( L SFA to anterior tibial bypass graft ). Left leg is extremely painful. More recently he has been on his feet more, which has been causing worsening edema 

In [ ]:
print(list_test[50])

<?xml version="1.0" encoding="UTF-8" ?>
<PatientMatching>
<TEXT><![CDATA[

Record date: 2098-04-01



Mr. Fredrick is a 48-year-old man status post right middle lobectomy on September 24, 2096, for T1N0 carcinoid.  He has had no complaints, since that time, and denies any stridor, wheezing, productive cough, shortness of breath, hemoptysis or fever.  He has gained several pounds, but is otherwise feeling well.

On physical exam today, the patient appears quite well.  HEENT examination shows no JVD or thyromegaly.  He has no palpable cervical, supraclavicular or axillary adenopathy.  His blood pressure today is 130/88, with a regular heart rate and rhythm of 89.  S1 and S2 are within normal limits; and there is no murmur, rub or gallop.  His resting O2 saturation is 100% on room air, with a respiratory rate of 20 per minute.  His temperature is 98.6, and his weight is 232 pounds.  His chest is clear to auscultation and percussion, with good excursion and no change in tactile fremitus or

In [ ]:
print(summaries_original_compact_biobert[50])

<? xml version = " 1. 0 " encoding = " UTF - 8 "? > < PatientMatching > < TEXT > <! [ CDATA [ Record date : 2098 - 04 - 01 Mr. Fredrick is a 48 - year - old man status post right middle lobectomy on September 24, 2096, for T1N0 carcinoid. He has had no complaints, since that time, and denies any stridor, wheezing, productive cough, shortness of breath, hemoptysis or fever. He has gained several pounds, but is otherwise feeling well. On physical exam today, the patient appears quite well. HEENT examination shows no JVD or thyromegaly. He has no palpable cervical, supraclavicular or axillary adenopathy. His blood pressure today is 130 / 88, with a regular heart rate and rhythm of 89. S1 and S2 are within normal limits ; and there is no murmur, rub or gallop. His resting O2 saturation is 100 % on room air, with a respiratory rate of 20 per minute. His temperature is 98. 6, and his weight is 232 pounds. His chest is clear to auscultation and percussion, with good excursion and no change in

In [ ]:
print(summaries_trained_compact_biobert[50])

<? xml version = " 1. 0 " encoding = " UTF - 8 "? > < PatientMatching > < TEXT > <! [ CDATA [ Record date : 2098 - 04 - 01 Mr. Fredrick is a 48 - year - old man status post right middle lobectomy on September 24, 2096, for T1N0 carcinoid. He has had no complaints, since that time, and denies any stridor, wheezing, productive cough, shortness of breath, hemoptysis or fever. He has gained several pounds, but is otherwise feeling well. On physical exam today, the patient appears quite well. HEENT examination shows no JVD or thyromegaly. He has no palpable cervical, supraclavicular or axillary adenopathy. His blood pressure today is 130 / 88, with a regular heart rate and rhythm of 89. S1 and S2 are within normal limits ; and there is no murmur, rub or gallop. His resting O2 saturation is 100 % on room air, with a respiratory rate of 20 per minute. His temperature is 98. 6, and his weight is 232 pounds. His chest is clear to auscultation and percussion, with good excursion and no change in

In [ ]:
print(summaries_original_ClinicalBioBert[1])

admission date : [ * * 2207 - 8 - 16 * * ] discharge date : [ * * 2207 - 9 - 2 * * ] date of birth : [ * * 2150 - 9 - 2 * * ] sex : m service : surgery allergies : no known allergies / adverse drug reactions attending : [ * * first name3 ( lf ) 1234 * * ] chief complaint : severe lower extremity lymphedema with superimposed cellulitis major surgical or invasive procedure : [ * * 2207 - 8 - 24 * * ] : left below - knee amputation ( circular guillotine ). [ * * 2207 - 8 - 28 * * ] : re - amputation and closure of left below - knee amputation. history of present illness : 56yo male with a history of dm2, cad s / p cabg, peripheral vascular disease, and prior history of chronic lymphedema was admitted on [ * * 2207 - 8 - 16 * * ] for lle cellulitis. his left lower extremity lymphedema started after vascular surgery in [ * * 2205 * * ] ( l sfa to anterior tibial bypass graft ). left leg is extremely painful. more recently he has been on his feet more, which has been causing worsening edema 

In [ ]:
print(summaries_trained_ClinicalBioBert[1])

admission date : [ * * 2207 - 8 - 16 * * ] discharge date : [ * * 2207 - 9 - 2 * * ] date of birth : [ * * 2150 - 9 - 2 * * ] sex : m service : surgery allergies : no known allergies / adverse drug reactions attending : [ * * first name3 ( lf ) 1234 * * ] chief complaint : severe lower extremity lymphedema with superimposed cellulitis major surgical or invasive procedure : [ * * 2207 - 8 - 24 * * ] : left below - knee amputation ( circular guillotine ). [ * * 2207 - 8 - 28 * * ] : re - amputation and closure of left below - knee amputation. history of present illness : 56yo male with a history of dm2, cad s / p cabg, peripheral vascular disease, and prior history of chronic lymphedema was admitted on [ * * 2207 - 8 - 16 * * ] for lle cellulitis. his left lower extremity lymphedema started after vascular surgery in [ * * 2205 * * ] ( l sfa to anterior tibial bypass graft ). left leg is extremely painful. more recently he has been on his feet more, which has been causing worsening edema 

In [ ]:
print(summaries_original_ClinicalBioBert[50])

<? xml version = " 1. 0 " encoding = " utf - 8 "? > < patientmatching > < text > <! [ cdata [ record date : 2098 - 04 - 01 mr. fredrick is a 48 - year - old man status post right middle lobectomy on september 24, 2096, for t1n0 carcinoid. he has had no complaints, since that time, and denies any stridor, wheezing, productive cough, shortness of breath, hemoptysis or fever. he has gained several pounds, but is otherwise feeling well. on physical exam today, the patient appears quite well. heent examination shows no jvd or thyromegaly. he has no palpable cervical, supraclavicular or axillary adenopathy. his blood pressure today is 130 / 88, with a regular heart rate and rhythm of 89. s1 and s2 are within normal limits ; and there is no murmur, rub or gallop. his resting o2 saturation is 100 % on room air, with a respiratory rate of 20 per minute. his temperature is 98. 6, and his weight is 232 pounds. his chest is clear to auscultation and percussion, with good excursion and no change in

In [ ]:
print(summaries_trained_ClinicalBioBert[50])

<? xml version = " 1. 0 " encoding = " utf - 8 "? > < patientmatching > < text > <! [ cdata [ record date : 2098 - 04 - 01 mr. fredrick is a 48 - year - old man status post right middle lobectomy on september 24, 2096, for t1n0 carcinoid. he has had no complaints, since that time, and denies any stridor, wheezing, productive cough, shortness of breath, hemoptysis or fever. he has gained several pounds, but is otherwise feeling well. on physical exam today, the patient appears quite well. heent examination shows no jvd or thyromegaly. he has no palpable cervical, supraclavicular or axillary adenopathy. his blood pressure today is 130 / 88, with a regular heart rate and rhythm of 89. s1 and s2 are within normal limits ; and there is no murmur, rub or gallop. his resting o2 saturation is 100 % on room air, with a respiratory rate of 20 per minute. his temperature is 98. 6, and his weight is 232 pounds. his chest is clear to auscultation and percussion, with good excursion and no change in

In [ ]:
print(summaries_original_SciBert[1])

admission date : [ * * 2207 - 8 - 16 * * ] discharge date : [ * * 2207 - 9 - 2 * * ] date of birth : [ * * 2150 - 9 - 2 * * ] sex : m service : surgery allergies : no known allergies / adverse drug reactions attending : [ * * first name3 ( lf ) 1234 * * ] chief complaint : severe lower extremity lymphedema with superimposed cellulitis major surgical or invasive procedure : [ * * 2207 - 8 - 24 * * ] : left below - knee amputation ( circular guillotine ). [ * * 2207 - 8 - 28 * * ] : re - amputation and closure of left below - knee amputation. history of present illness : 56yo male with a history of dm2, cad s / p cabg, peripheral vascular disease, and prior history of chronic lymphedema was admitted on [ * * 2207 - 8 - 16 * * ] for lle cellulitis. his left lower extremity lymphedema started after vascular surgery in [ * * 2205 * * ] ( l sfa to anterior tibial bypass graft ). left leg is extremely painful. more recently he has been on his feet more, which has been causing worsening edema 

In [ ]:
print(summaries_trained_SciBert[1])

admission date : [ * * 2207 - 8 - 16 * * ] discharge date : [ * * 2207 - 9 - 2 * * ] date of birth : [ * * 2150 - 9 - 2 * * ] sex : m service : surgery allergies : no known allergies / adverse drug reactions attending : [ * * first name3 ( lf ) 1234 * * ] chief complaint : severe lower extremity lymphedema with superimposed cellulitis major surgical or invasive procedure : [ * * 2207 - 8 - 24 * * ] : left below - knee amputation ( circular guillotine ). [ * * 2207 - 8 - 28 * * ] : re - amputation and closure of left below - knee amputation. history of present illness : 56yo male with a history of dm2, cad s / p cabg, peripheral vascular disease, and prior history of chronic lymphedema was admitted on [ * * 2207 - 8 - 16 * * ] for lle cellulitis. his left lower extremity lymphedema started after vascular surgery in [ * * 2205 * * ] ( l sfa to anterior tibial bypass graft ). left leg is extremely painful. more recently he has been on his feet more, which has been causing worsening edema 

In [ ]:
print(summaries_original_SciBert[50])

<? xml version = " 1. 0 " encoding = " utf - 8 "? > < patientmatching > < text > <! [ cdata [ record date : 2098 - 04 - 01 mr. fredrick is a 48 - year - old man status post right middle lobectomy on september 24, 2096, for t1n0 carcinoid. he has had no complaints, since that time, and denies any stridor, wheezing, productive cough, shortness of breath, hemoptysis or fever. he has gained several pounds, but is otherwise feeling well. on physical exam today, the patient appears quite well. heent examination shows no jvd or thyromegaly. he has no palpable cervical, supraclavicular or axillary adenopathy. his blood pressure today is 130 / 88, with a regular heart rate and rhythm of 89. s1 and s2 are within normal limits ; and there is no murmur, rub or gallop. his resting o2 saturation is 100 % on room air, with a respiratory rate of 20 per minute. his temperature is 98. 6, and his weight is 232 pounds. his chest is clear to auscultation and percussion, with good excursion and no change in

In [ ]:
print(summaries_trained_SciBert[50])

<? xml version = " 1. 0 " encoding = " utf - 8 "? > < patientmatching > < text > <! [ cdata [ record date : 2098 - 04 - 01 mr. fredrick is a 48 - year - old man status post right middle lobectomy on september 24, 2096, for t1n0 carcinoid. he has had no complaints, since that time, and denies any stridor, wheezing, productive cough, shortness of breath, hemoptysis or fever. he has gained several pounds, but is otherwise feeling well. on physical exam today, the patient appears quite well. heent examination shows no jvd or thyromegaly. he has no palpable cervical, supraclavicular or axillary adenopathy. his blood pressure today is 130 / 88, with a regular heart rate and rhythm of 89. s1 and s2 are within normal limits ; and there is no murmur, rub or gallop. his resting o2 saturation is 100 % on room air, with a respiratory rate of 20 per minute. his temperature is 98. 6, and his weight is 232 pounds. his chest is clear to auscultation and percussion, with good excursion and no change in

In [ ]:
print(summaries_original_biogpt[1])

Admission Date: [* * 2207-8-16 * *] Discharge Date: [* * 2207-9-2 * *] Date of Birth: [* * 2150-9-2 * *] Sex: M Service: SURGERY Allergies: No Known Allergies / Adverse Drug Reactions Attending: [* * First Name3 (LF) 1234 * *] Chief Complaint: Severe lower extremity lymphedema with superimposed cellulitis Major Surgical or Invasive Procedure: [* * 2207-8-24 * *]: Left below-knee amputation (circular guillotine). [* * 2207-8-28 * *]: Re-amputation and closure of left below-knee amputation. History of Present Illness: 56yo male with a history of DM2, CAD s / p CABG, peripheral vascular disease, and prior history of chronic lymphedema was admitted on [* * 2207-8-16 * *] for LLE cellulitis. His left lower extremity lymphedema started after vascular surgery in [* * 2205 * *] (L SFA to anterior tibial bypass graft). Left leg is extremely painful. More recently he has been on his feet more, which has been causing worsening edema and pain. He has occasional serous drainage from left leg. He wa

In [ ]:
print(summaries_trained_biogpt[1])

Admission Date: [* * 2207-8-16 * *] Discharge Date: [* * 2207-9-2 * *] Date of Birth: [* * 2150-9-2 * *] Sex: M Service: SURGERY Allergies: No Known Allergies / Adverse Drug Reactions Attending: [* * First Name3 (LF) 1234 * *] Chief Complaint: Severe lower extremity lymphedema with superimposed cellulitis Major Surgical or Invasive Procedure: [* * 2207-8-24 * *]: Left below-knee amputation (circular guillotine). [* * 2207-8-28 * *]: Re-amputation and closure of left below-knee amputation. History of Present Illness: 56yo male with a history of DM2, CAD s / p CABG, peripheral vascular disease, and prior history of chronic lymphedema was admitted on [* * 2207-8-16 * *] for LLE cellulitis. His left lower extremity lymphedema started after vascular surgery in [* * 2205 * *] (L SFA to anterior tibial bypass graft). Left leg is extremely painful. More recently he has been on his feet more, which has been causing worsening edema and pain. He has occasional serous drainage from left leg. He wa

In [ ]:
print(summaries_original_biogpt[50])

<? xml version = "1.0" encoding = "UTF-8"? > < PatientMatching > < TEXT > <! [CDATA [Record date: 2098-04-01 Mr. Fredrick is a 48-year-old man status post right middle lobectomy on September 24, 2096, for T1N0 carcinoid. He has had no complaints, since that time, and denies any stridor, wheezing, productive cough, shortness of breath, hemoptysis or fever. He has gained several pounds, but is otherwise feeling well. On physical exam today, the patient appears quite well. HEENT examination shows no JVD or thyromegaly. He has no palpable cervical, supraclavicular or axillary adenopathy. His blood pressure today is 130 / 88, with a regular heart rate and rhythm of 89. S1 and S2 are within normal limits; and there is no murmur, rub or gallop. His resting O2 saturation is 100% on room air, with a respiratory rate of 20 per minute. His temperature is 98.6, and his weight is 232 pounds. His chest is clear to auscultation and percussion, with good excursion and no change in tactile fremitus or 

In [ ]:
print(summaries_trained_biogpt[50])

<? xml version = "1.0" encoding = "UTF-8"? > < PatientMatching > < TEXT > <! [CDATA [Record date: 2098-04-01 Mr. Fredrick is a 48-year-old man status post right middle lobectomy on September 24, 2096, for T1N0 carcinoid. He has had no complaints, since that time, and denies any stridor, wheezing, productive cough, shortness of breath, hemoptysis or fever. He has gained several pounds, but is otherwise feeling well. On physical exam today, the patient appears quite well. HEENT examination shows no JVD or thyromegaly. He has no palpable cervical, supraclavicular or axillary adenopathy. His blood pressure today is 130 / 88, with a regular heart rate and rhythm of 89. S1 and S2 are within normal limits; and there is no murmur, rub or gallop. His resting O2 saturation is 100% on room air, with a respiratory rate of 20 per minute. His temperature is 98.6, and his weight is 232 pounds. His chest is clear to auscultation and percussion, with good excursion and no change in tactile fremitus or 